In [1]:
import os
import re
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def moving_average(data, window):
    padding_num = int(np.floor(window / 2))
    start_padding = data[:padding_num].mean().repeat(padding_num)
    end_padding = data[len(data)-padding_num:].mean().repeat(padding_num)

    values = np.concatenate([start_padding, data, end_padding])

    ret = np.cumsum(values, dtype=float)
    ret[window:] = ret[window:] - ret[:-window]
    return ret[window - 1:] / window


class MovingAverager:
    def __init__(self, windows):
        if isinstance(windows, List):
            self.windows = windows
        elif isinstance(windows, int):
            self.windows = [windows]
        self.max_window = max(self.windows)
    
    def execute(self, data):
        assert len(data) >= self.max_window
        
        result = np.zeros(len(data))

        for window in self.windows:
            result += moving_average(data=data, window=window)
        result /= len(self.windows)

        return result

In [3]:
def read_from_folder(folder):
    acc_df = pd.read_csv(os.path.join(folder, 'Accelerometer.csv'))
    lin_acc_df = pd.read_csv(os.path.join(folder, 'Linear Accelerometer.csv'))
    mag_df = pd.read_csv(os.path.join(folder, 'Magnetometer.csv'))
    gyro_df = pd.read_csv(os.path.join(folder, 'Gyroscope.csv'))
    
    total_df = pd.concat([acc_df, gyro_df, lin_acc_df, mag_df], axis=1)
    return total_df


def rename_data(df):
#     new_names = ['acc_times', 'acc_x', 'acc_y', 'acc_z', 'gyo_times', 'gyo_x', 'gyo_y', 'gyo_z', 'lin_acc_times', 'lin_acc_x', 'lin_acc_y', 'lin_acc_z', 'mag_times', 'mag_x', 'mag_y', 'mag_z']
    new_names = ['times', 'acc_x', 'acc_y', 'acc_z', 'gyo_times', 'gyo_x', 'gyo_y', 'gyo_z', 'lin_acc_times', 'lin_acc_x', 'lin_acc_y', 'lin_acc_z', 'mag_times', 'mag_x', 'mag_y', 'mag_z']
    df.columns = new_names
    return df


def gen_total_mag(df: pd.DataFrame):
    df['mag_t'] = np.sqrt((df['mag_x'] ** 2) + (df['mag_y'] ** 2) + (df['mag_z'] ** 2))
    return df


def smooth(df):
    for column in df.columns:
        if 'time' in column:
            continue
        df[column] = moving_average(df[column].to_numpy(), 11)
    return df


def select_data(df):
    return df[['times', 'acc_x', 'acc_y', 'acc_z', 'lin_acc_x', 'lin_acc_y', 'lin_acc_z', 'gyo_x', 'gyo_y', 'gyo_z', 'mag_x', 'mag_y', 'mag_z', 'mag_t']]


def bound_range(df, datapoint_per_second=20):
    start = int(datapoint_per_second * 1)
    end = int(len(df) - datapoint_per_second * 1)
    
    return df.iloc[start:end].reset_index(drop=True)


def load_data(folder):
    total_df = read_from_folder(folder)
    total_df = total_df[:-5]  # remove last None value row
    total_df = rename_data(total_df)
    
    total_df = gen_total_mag(total_df)
    total_df = smooth(total_df)
    
    total_df = select_data(total_df)
    total_df = bound_range(total_df)
    
    return total_df

In [4]:
all_data = {}

for user_folder in sorted(glob.glob('./R403_trajectory/user*')):
    user = user_folder.split('/')[-1]
    all_data[user] = {}
    
    for path_folder in sorted(glob.glob(os.path.join(user_folder, 'Path*'))):   
        path = path_folder.split('/')[-1]

        temp = {}
        for i, posture_folder in enumerate(sorted(glob.glob(os.path.join(path_folder, '*[!zip]')))):
            posture = posture_folder.split('/')[-1]

            df = load_data(posture_folder)
            temp[posture] = df.to_numpy()
                
        all_data[user][path] = temp

In [178]:
for path_name in all_data['User1'].keys():
    path_point = np.array(all_path[path_name])
    
    for posture in all_data['User1'][path_name].keys():
        
        data = all_data['User1'][path_name][posture]
        
        split_index = [i / (len(path_point) - 1) * (len(data) - 1) for i in range((len(path_point)))]
        split_index = np.round(split_index).astype(int)
        
        sample_point = data[split_index]
        all_data['User1'][path_name][posture] = np.concatenate((path_point, sample_point), axis=-1)

In [179]:
np.save('./User1_data', all_data['User1'])

In [181]:
np.save('./all_User_data', all_data)

In [156]:
all_path = {}

In [174]:
path = []
for i in range(9):
    path.append((i, 8))
for i in range(8, 0, -1):
    path.append((9, i))
for i in range(9, -1, -1):
    path.append((i, 0))

In [175]:
path, len(path)

([(0, 8),
  (1, 8),
  (2, 8),
  (3, 8),
  (4, 8),
  (5, 8),
  (6, 8),
  (7, 8),
  (8, 8),
  (9, 8),
  (9, 7),
  (9, 6),
  (9, 5),
  (9, 4),
  (9, 3),
  (9, 2),
  (9, 1),
  (9, 0),
  (8, 0),
  (7, 0),
  (6, 0),
  (5, 0),
  (4, 0),
  (3, 0),
  (2, 0),
  (1, 0),
  (0, 0)],
 27)

In [176]:
all_path['path5'] = path

In [177]:
np.save('./User1_path', all_path)